# Retrieval Data dengan Keywords Embedding

Konsep dasar dari retrieval berbasis keyword adalah mengekstrak inti poin dari sebuah kalimat (keyword), kemudian melakukan retrieval menggunakan keyword tersebut. Pendekatan ini diharapkan dapat mengurangi `bias` dan membuat hasil retrieval menjadi lebih `fokus` serta `relevan`.

## Cara melakukannya?

1. **Cleaning data text** - lakukan preprocessing dasar (lowercase, hapus karakter khusus, dll.).
2. **Stopword** (optional) - buang kata-kata umum yang tidak memiliki makna penting.
3. **Tokenize sentence** - bisa menggunakan unigram atau n-gram sesuai kebutuhan.
4. **Embedding** – setiap token dan kalimat diubah menjadi embedding.
5. **Similarity check** – hitung cosine similarity antara setiap token dengan embedding kalimat.
6. **Pilih top-N keyword** – ambil token dengan skor tertinggi, ulangi untuk setiap kalimat.

Hasil akhirnya, kita akan mendapatkan dokumen dengan top-N keywords yang paling representatif. Setelah keyword diperoleh, lakukan embedding kembali pada keyword tersebut dan simpan sebagai `keywords_embedding`.

📌 Pada contoh ini saya menggunakan data QnA kesehatan dari Alodokter, tapi konsepnya bisa dengan mudah diterapkan pada berbagai jenis dokumen lain.

In [1]:
import pandas as pd

df = pd.read_csv("../alo_qna_clean.csv")
df.head()

,title,question,answer,doctor_name,tag,url
0,Bagaimana cara menghilangkan kurap dengan cepat?,dokter di lengan dan pundak kiri saya ada kura...,"Alo, terimakasih atas pertanyaannya.\n\nRuam m...",dr. Nadia Nurotul Fuadah,infeksi-jamur kurap,https://www.alodokter.com/komunitas/topic/baga...
1,Pake obat apa untuk mengatasi jerawat hormonal?,"hallo dokter, dok saat menjelang haid saya pas...","Alo, selamat siang\nKemunculan jerawat saat ha...",dr. Riza Marlina,jerawat,https://www.alodokter.com/komunitas/topic/pake...
2,Cara membersihkan telinga anak dirumah,"alodokter, anak saya telinganya sering mengelu...","Alo, selamat siang\nTelinga gatal bisa disebab...",dr. Riza Marlina,kebersihan kotoran-telinga,https://www.alodokter.com/komunitas/topic/cara...
3,Solusi mengatasi bayi usia 9 bulan susah makan,"alodokter, saya mau bertanya, bayi saya usia 9...","Alo, selamat siang\nBayi susah makan disebabka...",dr. Riza Marlina,nutrisi-bayi,https://www.alodokter.com/komunitas/topic/solu...
4,Apa yang harus dilakukan ketika kaki kram saat...,"permisi dok, dokter kalau mengatasi kaki suka ...","Alo, selamat siang\nKaki kram dan seperti tert...",dr. Riza Marlina,kram,https://www.alodokter.com/komunitas/topic/apa-...


In [2]:
len(df)

288105

In [3]:
# get 5000 data
df = df.head(5000)
len(df)

5000

Kalau diliat data saya memiliki sekitar `+280.000` data qna, dan saya hanya akan coba pakai sekitar `5000` data saja. Agar secara komputasi nanti tidak terlalu lama, dan GPU yang saya gunakan adalah `NVIDIA GeForce RTX 3060 (12GB VRAM)`.

In [4]:
# check sample question
df["question"][10]

'alodokter, dok saya sudah seminggu lebih merasakan keputihan serta vagina gatal gatal dan muncul keputihan yg menggumpal berwarna putih-kuning tapi tidak berbau, serta ketika gatal terasa ingin buang air kecil, itu kenapa ya? dan obatnya apa? apakah dijual di apotik??'

## Bikin Fungsi Cleaning Text

Fungsi ini bisa kalian sesuaikan dengan data kalian, karena setiap data mungkin ada perlakuan yang berbeda.

In [5]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words("indonesian"))

custom_stopwords = {"alodokter", "alo", "hallo", "halo", "dok", "dokter", "apotik"}
stop_words = stop_words.union(custom_stopwords)

def clean_text(text: str):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(text)
    tokens = [t for t in tokens if t not in stop_words]
    
    return tokens

sample = df["question"][10]
print(clean_text(sample))

['seminggu', 'merasakan', 'keputihan', 'vagina', 'gatal', 'gatal', 'muncul', 'keputihan', 'yg', 'menggumpal', 'berwarna', 'putihkuning', 'berbau', 'gatal', 'buang', 'air', 'ya', 'obatnya', 'dijual']


[nltk_data] Downloading package punkt to /var/www/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /var/www/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Bikin Token Unigram dan Bigram

Kenapa saya juga menggunakan Bigram juga? Saya ingin memiliki kombinasi yang lebih banyak ketimbang hanya menggunakan Unigram.

Contoh:

- "Budi pergi ke pasar" = ["budi", "pergi", "ke", "pasar"] (Unigram)
- "Budi pergi ke pasar" = ["budi", "budi pergi", "pergi", "pergi ke", ...] (Bigram)

Tujuan nya hanya untuk mendapatkan lebih banyak kombinasi keyword agar lebih bervariasi. Kalau kalian ingin kombinasi sampai 3 token pun silahkan, tidak ada larangan untuk mencobanya!

In [6]:
import nltk
from nltk import word_tokenize

token = clean_text(df["question"][10])

token_ngram = []

for i in range(len(token)):
    token_ngram.append(token[i]) # get unigram
    
    if i < len(token) - 1:
        token_ngram.append(token[i] + " " + token[i+1]) # get bigram

print("LENGTH TOKEN BEFORE BIGRAM: ", len(token))
print("LENGTH TOKEN AFTER BIGRAM: ", len(token_ngram))
token_ngram[:8]

LENGTH TOKEN BEFORE BIGRAM:  19
LENGTH TOKEN AFTER BIGRAM:  37


['seminggu',
 'seminggu merasakan',
 'merasakan',
 'merasakan keputihan',
 'keputihan',
 'keputihan vagina',
 'vagina',
 'vagina gatal']

## Bikin Keyword sekaligus mencari cosine similarity paling besar

In [7]:
from tqdm.auto import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

sentence_embedding = sentence_model.encode(df["question"][10])

rows = {"keyword": [], "cosine_similarity": []}

for token in tqdm(token_ngram):
    token_embedding = sentence_model.encode(token)
    distance = cosine_similarity([sentence_embedding], [token_embedding])
    rows["keyword"].append(token)
    rows["cosine_similarity"].append(distance.item())

df_result = pd.DataFrame(rows)

df_result["keyword_lower"] = df_result["keyword"].str.lower()
df_result = df_result.drop_duplicates(subset="keyword_lower", keep="first")
df_result = df_result.drop(columns="keyword_lower")

df_result = df_result.sort_values(by="cosine_similarity", ascending=False).reset_index(drop=True)

print(df_result.head(10))

2025-09-03 15:28:17.173394: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756888097.196156 1688144 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756888097.203600 1688144 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756888097.221726 1688144 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756888097.221740 1688144 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756888097.221742 1688144 computation_placer.cc:177] computation placer alr

  0%|          | 0/37 [00:00<?, ?it/s]

               keyword  cosine_similarity
0  merasakan keputihan           0.675337
1     keputihan vagina           0.660185
2     muncul keputihan           0.649460
3         keputihan yg           0.647167
4            keputihan           0.635910
5         vagina gatal           0.607791
6               vagina           0.531140
7          gatal buang           0.500971
8          gatal gatal           0.480090
9         gatal muncul           0.458396


In [8]:
", ".join(df_result["keyword"][:3].values) # get top 3 and merge into one string

'merasakan keputihan, keputihan vagina, muncul keputihan'

## Gabungkan Semua Proses

In [9]:
df_keyword = {"document": [], "keywords": []}

# function to proccess unigram bigram
def make_unigram_bigram(tokens):
    token_ngram = []

    for i in range(len(tokens)):
        # unigram
        token_ngram.append(tokens[i])

        # bigram (selama masih ada token berikutnya)
        if i < len(tokens) - 1:
            token_ngram.append(tokens[i] + " " + tokens[i+1])

    return token_ngram


for data in tqdm(df["question"], desc="Document Process"):
    sentence_embedding = sentence_model.encode(data) # embedding sentence

    # Cleaning token
    tokens = clean_text(data)
    if not tokens:
        df_keyword["document"].append(data)
        df_keyword["keywords"].append("")
        continue
        
    # get unigram bigram
    tokens = make_unigram_bigram(tokens)

    token_embeddings = sentence_model.encode(tokens) # embedding each token
    sims = cosine_similarity(token_embeddings, sentence_embedding.reshape(1, -1)).ravel() # calculate similarity beetwen token and sentence

    df_result = pd.DataFrame({
        "keyword": tokens,
        "cosine_similarity": sims
    })

    df_result["keyword_lower"] = df_result["keyword"].str.lower() # into lower
    df_result = df_result.drop_duplicates(subset="keyword_lower", keep="first") # drop duplicate
    df_result = df_result.drop(columns="keyword_lower") # drop column keyword_lower

    df_result = df_result.sort_values(by="cosine_similarity", ascending=False).reset_index(drop=True) # sorting high value on top

    keywords = ", ".join(df_result["keyword"][:4].values) # get 4 top sim

    df_keyword["document"].append(data)
    df_keyword["keywords"].append(keywords)

df = pd.DataFrame(df_keyword)

Document Process:   0%|          | 0/5000 [00:00<?, ?it/s]

In [10]:
df.head()

,document,keywords
0,dokter di lengan dan pundak kiri saya ada kura...,"gatal risih, kurapnya gatal, gatal, keputihan ..."
1,"hallo dokter, dok saat menjelang haid saya pas...","jerawat hormonal, mengobati jerawat, menjelang..."
2,"alodokter, anak saya telinganya sering mengelu...","kotoran telinga, telinga kisaran, telinga, tel..."
3,"alodokter, saya mau bertanya, bayi saya usia 9...","bayi usia, bayi, susah makan, berat badan"
4,"permisi dok, dokter kalau mengatasi kaki suka ...","pake obat, obat, mengatasi kaki, obat ya"


# Cara Pakai

Dari hasil dataframe question dengan keyword, kita akan lakukan embedding hanya pada keywords saja. Dan bikin kolom baru dengan nama `keywords_embedding`. Setelah semua ready kita akan bikin fungsi untuk testing nya.

In [12]:
docs_keywords = {"document": [], "keywords": [], "keywords_embedding": []}
for text in df["keywords"]:
    
    keywords_emb = sentence_model.encode(text)
    
    docs_keywords["document"].append(text)
    docs_keywords["keywords"].append(text)
    docs_keywords["keywords_embedding"].append(keywords_emb)

df_keywords = pd.DataFrame(docs_keywords)

In [13]:
df_keywords.head()

,document,keywords,keywords_embedding
0,"gatal risih, kurapnya gatal, gatal, keputihan ...","gatal risih, kurapnya gatal, gatal, keputihan ...","[0.0018438789, -0.122727856, -0.014446956, -0...."
1,"jerawat hormonal, mengobati jerawat, menjelang...","jerawat hormonal, mengobati jerawat, menjelang...","[0.09429207, -0.031596616, -0.017314112, -0.18..."
2,"kotoran telinga, telinga kisaran, telinga, tel...","kotoran telinga, telinga kisaran, telinga, tel...","[-0.06866116, -0.11226666, -0.015629586, -0.01..."
3,"bayi usia, bayi, susah makan, berat badan","bayi usia, bayi, susah makan, berat badan","[-0.0060298634, 0.00986947, -0.013554898, -0.0..."
4,"pake obat, obat, mengatasi kaki, obat ya","pake obat, obat, mengatasi kaki, obat ya","[0.04805321, -0.035581518, -0.018161263, -0.05..."


In [14]:
def search_documents(query, top_k=5, top_n=4):
    
    q_emb = sentence_model.encode(query)

    similarities = []
    for doc_emb in df_keywords["keywords_embedding"]:
        sim = cosine_similarity([q_emb], [doc_emb])[0][0]
        similarities.append(sim)

    result = df_keywords.copy()
    result["similarity"] = similarities
    result = result.sort_values(by="similarity", ascending=False).head(top_k)

    return result[["document", "keywords", "similarity"]]

In [16]:
query = "saya mengalami batuk-batuk sejak awal bulan ini, kenapa yaa?"
result_df = search_documents(query, top_k=5)

display(result_df)

,document,keywords,similarity
2440,"ilang batuk, batuk berdahak, tenggorokan gatal...","ilang batuk, batuk berdahak, tenggorokan gatal...",0.866283
4317,"konsultasi batuk, gatal batuk, batuk seminggus...","konsultasi batuk, gatal batuk, batuk seminggus...",0.842918
3560,"mengalami batuk, batuk, seminggu batuk, batuk yg","mengalami batuk, batuk, seminggu batuk, batuk yg",0.831456
3006,"sebulan batuk, minum batuk, batuk, batuk tdk","sebulan batuk, minum batuk, batuk, batuk tdk",0.829194
2319,"sakit tenggorokan, tenggorokan ternggorokan, b...","sakit tenggorokan, tenggorokan ternggorokan, b...",0.827036


# Library Support

Sebenarnya sudah ada library yang mendukung ekstraksi keyword dengan embedding. Salah satunya adalah `KeyBERT`, yang memanfaatkan model BERT untuk mendapatkan keyword dari sebuah kalimat. Bahkan, kita bisa langsung mengatur ukuran n-gram sesuai kebutuhan.

In [17]:
from keybert import KeyBERT

doc = """
         Supervised learning is the machine learning task of learning a function that
         maps an input to an output based on example input-output pairs. It infers a
         function from labeled training data consisting of a set of training examples.
         In supervised learning, each example is a pair consisting of an input object
         (typically a vector) and a desired output value (also called the supervisory signal).
         A supervised learning algorithm analyzes the training data and produces an inferred function,
         which can be used for mapping new examples. An optimal scenario will allow for the
         algorithm to correctly determine the class labels for unseen instances. This requires
         the learning algorithm to generalize from the training data to unseen situations in a
         'reasonable' way (see inductive bias).
      """

kw_model = KeyBERT(model=sentence_model)
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(1, 2), stop_words='english')
keywords

[('supervised learning', 0.8006),
 ('supervised', 0.6602),
 ('called supervisory', 0.6286),
 ('learning machine', 0.6113),
 ('supervisory', 0.5946)]